# MPI parallelisation

We will need to install two packages:

- The python MPI package:

```
conda install mpi4py
```

- The actual MPI library:

```
conda install openmpi
```

### References: 
https://mpi4py.readthedocs.io/en/stable/tutorial.html

https://education.molssi.org/parallel-programming/03-distributed-examples-mpi4py/index.html


## 1. Basic python script:

Let's create a .py script that writes "Hello World":


```python
if __name__ == "__main__":

    print("Hello World!")
    
```

To execute it in a terminal:

```
wladimir$ python example_mpi.py
1.py 

Hello World!
```